# core

> Main codebase for fetching and saving RSS feeds

In [ ]:
# | default_exp core

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export

import json
import logging
from pathlib import Path
from typing import *

import fire
import reader
import rich
from bs4 import BeautifulSoup
from rich import print
from rich.console import Console
from rich.panel import Panel
from rich.text import Text

console = Console()

logger = logging.getLogger(__name__)


def strip_html(s: str):
    s = BeautifulSoup(s, "html.parser")
    return s.text

In [ ]:
# | export


class Feed:
    """RSS feed class"""

    def __init__(self, url: str, name: str = None, tags: List[str] = []):
        self.url = url
        self.name = name
        self.tags = tags

    def add_tag(self, tag: str):
        if tag not in self.tags:
            self.tags.append(tag)

    def remove_tag(self, tag: str):
        if tag in self.tags:
            self.tags.remove(tag)

In [ ]:
# | export


class PyNewsReader:
    def __init__(self, feeds=List[Feed]):
        self._dbfolder = Path().home() / ".cache/pynewsreader"
        if not self._dbfolder.exists():
            self._dbfolder.mkdir(parents=True)

        # If custom feed names exist, read them in
        feed_names_path = self._dbfolder / "feed_names.json"
        if feed_names_path.exists():
            with open(self._dbfolder / "feed_names.json", "rt") as myfile:
                self._feed_names = json.load(myfile)
        else:
            self._feed_names = {}

        # If title blacklist exists, read it in
        title_blacklist_path = self._dbfolder / "title_blacklist.json"
        if title_blacklist_path.exists():
            with open(self._dbfolder / "title_blacklist.json", "rt") as myfile:
                self._title_blacklist = json.load(myfile)
        else:
            self._title_blacklist = []

        self._reader = reader.make_reader(
            self._dbfolder / "db.sqlite",
            plugins=["reader.enclosure_dedupe", "reader.entry_dedupe"],
        )

        # If title whitelist exists, read it in
        title_whitelist_path = self._dbfolder / "title_whitelist.json"
        if title_whitelist_path.exists():
            with open(self._dbfolder / "title_whitelist.json", "rt") as myfile:
                self._title_whitelist = json.load(myfile)
        else:
            self._title_whitelist = []

        self._reader = reader.make_reader(
            self._dbfolder / "db.sqlite",
            plugins=["reader.enclosure_dedupe", "reader.entry_dedupe"],
        )

        self._reader.enable_search()

    def _print_entries(
        self, entries: List[reader.Entry], mark_as_read: bool = True, limit: int = 10
    ):
        """Pretty print entries - supports reader.Reader.get_entries arguments"""
        displayed_links = set()
        for e in entries:
            if e.link in displayed_links:
                # Don't display duplicates
                self._reader.mark_entry_as_read(e)
            else:
                displayed_links.add(e.link)
                if e.published:
                    published_date = "Date: " + e.published.isoformat()[:10]
                else:
                    published_date = "Date: Unknown"
                if mark_as_read:
                    self._reader.mark_entry_as_read(e)

                feed_title = f"[bold]{self._get_feed_title(e.original_feed_url)}[/bold]"

                if e.important:
                    panel_body = ":exclamation_mark:"
                else:
                    panel_body = ""
                panel_body += f"Title: [bold]{e.title}[/bold]" + "\n"
                panel_body += str(published_date) + "\n\n"
                panel_body += strip_html(e.summary).strip() + "\n"

                console.print(
                    Panel(
                        panel_body,
                        title=feed_title,
                        subtitle=f"[link={e.link}]{e.link}[/link]",
                    )
                )
                console.print()
            if len(displayed_links) == limit:
                return

    def _search_to_entry(self, search_result):
        """For a given `reader` search result return the corresponding `reader.Entry`"""
        for i in self._reader.get_entries():
            if i.id == search_result.id and i.feed_url == search_result.feed_url:
                return i

    def _get_feed_title(self, url: str):
        """Get display title for pynewsreader feed"""
        if url in self._feed_names and self._feed_names[url] is not None:
            return self._feed_names[url]
        elif self._reader.get_feed(url).title:
            return self._reader.get_feed(url).title
        else:
            return self._reader.get_feed(url).url

    def _get_entries(
        self, important: bool = None, read: Union[None, bool] = None, limit: int = 10
    ) -> List[reader.Entry]:
        """Get entries in reader.Entry format"""
        return self._reader.get_entries(read=read, limit=limit, important=important)

    def _get_tags(self, entry: reader.Entry):
        """Get tags for a given entry"""
        return [i[0] for i in list(self._reader.get_tags(entry))]

    def _mark_matching_entries_as_read(self, match_strings: List):
        for i in self._reader.get_entries(read=False):
            for filter_string in match_strings:
                if filter_string in i.title:
                    print(f"Marking entry as read: {i.title}")
                    self._reader.mark_entry_as_read(i)
                if filter_string in i.link:
                    print(f"Marking entry as read: {i.title}")
                    self._reader.mark_entry_as_read(i)

    def _mark_matching_entries_as_important(self, match_strings: List):
        for i in self._reader.get_entries(read=False):
            for filter_string in match_strings:
                if filter_string in i.title:
                    print(f"Marking entry as important: {i.title}")
                    self._reader.mark_entry_as_important(i)

    def _add_to_blacklist(self, blacklist_string: str):
        """Add entry to blacklist"""
        if blacklist_string not in self._title_blacklist:
            self._title_blacklist.append(blacklist_string)
            with open(self._dbfolder / "title_blacklist.json", "wt") as myfile:
                json.dump(self._title_blacklist, myfile)

    def _remove_from_blacklist(self, blacklist_string: str):
        """Remove entry from blacklist"""
        if blacklist_string in self._title_blacklist:
            self._title_blacklist.remove(blacklist_string)
            with open(self._dbfolder / "title_blacklist.json", "wt") as myfile:
                json.dump(self._title_blacklist, myfile)

    def _add_to_whitelist(self, whitelist_string: str):
        """Add entry to whitelist"""
        if whitelist_string not in self._title_whitelist:
            self._title_whitelist.append(whitelist_string)
            with open(self._dbfolder / "title_whitelist.json", "wt") as myfile:
                json.dump(self._title_whitelist, myfile)
            for entry in self._get_entries(limit=None):
                if whitelist_string in entry.title:
                    self._reader.mark_entry_as_important(entry)

    def _remove_from_whitelist(self, whitelist_string: str):
        """Remove whitelist entry"""
        if whitelist_string in self._title_whitelist:
            self._title_whitelist.remove(whitelist_string)
            with open(self._dbfolder / "title_whitelist.json", "wt") as myfile:
                json.dump(self._title_whitelist, myfile)
            for entry in self._get_entries(limit=None):
                if whitelist_string in entry.title:
                    self._reader.mark_entry_as_unimportant(entry)

    def _mark_important(self, entry: reader.Entry = None):
        """Mark entry as important"""
        if entry is not None:
            reader.Reader.mark_entry_as_important(entry)

    def _mark_unimportant(self, entry: reader.Entry = None):
        """Mark entry as important"""
        if entry is not None:
            reader.Reader.mark_entry_as_unimportant(entry)

    def _add_tag(self, entry: reader.Entry, tag_key: str, tag_value: Dict = None):
        """Add tag to entry"""
        reader.Reader.set_tag(entry, tag_key, tag_value)

    def _remove_tag(self, entry: reader.Entry, tag_key: str):
        """Remove tag from entry"""
        self._reader.delete_tag(entry, tag_key)

    def add_feed(self, feed: Union[Feed, str]):
        """Add feed to pynewsreader

        Args:
            feed (Feed): pynewsreader Feed to add
        """
        if isinstance(feed, Feed):
            self._feed_names[feed.url] = feed.name
            self._reader.add_feed(feed.url, exist_ok=True)
        elif isinstance(feed, str):
            self._reader.add_feed(feed, exist_ok=True)
        else:
            raise Exception("Must be str or Feed type to add")

        # Save names to file
        with open(self._dbfolder / "feed_names.json", "wt") as myfile:
            json.dump(self._feed_names, myfile)

    def remove_feed(self, feed: Union[Feed, str]):
        """Remove feed from pynewsreader instance

        Args:
            feed (Union[Feed, str]): Feed to remove
        """

        if isinstance(feed, Feed):
            self._reader.delete_feed(feed.url)
        elif isinstance(feed, str):
            self._reader.delete_feed(feed)
        else:
            raise Exception(TypeError)

    def feeds(self):
        """List pynewsreader feeds

        Returns:
            List[str]: List of names of current pynewsreader feeds
        """
        return [self._get_feed_title(i.url) for i in self._reader.get_feeds()]

    def update(self):
        """Update feeds and search"""
        self._reader.update_feeds()
        self._reader.update_search()
        if len(self._title_blacklist) > 0:
            self._mark_matching_entries_as_read(self._title_blacklist)
        if len(self._title_whitelist) > 0:
            self._mark_matching_entries_as_important(self._title_whitelist)

    def show(
        self,
        limit: int = 6,
        read: bool = False,
        important: bool = None,
        mark_as_read: bool = True,
    ):
        """Pretty print entries

        Args:
            limit (int, optional): Number of entries to show. Defaults to 5.
            read (bool, optional): Show read entries (True), unread entries (False), or all entries (None). Defaults to None.
            mark_as_read (bool, optional): Mark displayed entries as read. Defaults to False.
        """
        self._print_entries(
            self._get_entries(read=read, important=important, limit=limit * 2),
            limit=limit,
            mark_as_read=mark_as_read,
        )

    def search(self, query: str, mark_as_read: bool = True, limit: int = 10):
        """Search entries and pretty print results

        Args:
            query (str): Search query
            mark_as_read (bool, optional): Mark results as read? Defaults to True.
        """
        self._print_entries(
            [self._search_to_entry(i) for i in self._reader.search_entries(query)],
            mark_as_read=mark_as_read,
            limit=limit,
        )

In [ ]:
r = PyNewsReader()

## Add Feeds

In [ ]:
# With names
for i in [
    Feed(url="https://ricochet.media/en/feed", name="Richochet Media"),
    Feed(url="https://thetyee.ca/rss2.xml", name="The Tyee"),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.topstories.rss",
        name="Toronto Star | Top Stories",
    ),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.news.investigations.rss",
        name="Toronto Star | Investigations",
    ),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.opinion.editorials.rss",
        name="Toronto Star | Editorials",
    ),
    Feed(url="https://www.macleans.ca/feed/", name="Macleans"),
]:
    r.add_feed(i)

# Use the default name from the feed
for i in [
    "https://rss.cbc.ca/lineup/topstories.xml",
    "https://rss.cbc.ca/lineup/world.xml",
    "https://rss.cbc.ca/lineup/canada.xml",
    "https://rss.cbc.ca/lineup/business.xml",
    "https://rss.cbc.ca/lineup/technology.xml",
    "https://www.cbc.ca/cmlink/rss-canada-ottawa",
    "https://thenarwhal.ca/feed/",
    "https://www.theguardian.com/science/rss",
    "https://www.theguardian.com/world/canada/rss",
]:
    r.add_feed(Feed(i))

## Update Feeds

In [ ]:
r.update()

## List Feeds

In [ ]:
r.feeds()

['Canada | The Guardian',
 'CBC | Business News',
 'CBC | Canada News',
 'CBC | Ottawa News',
 'CBC | Technology News',
 'CBC | Top Stories News',
 'CBC | World News',
 'Macleans',
 'Richochet Media',
 'Science | The Guardian',
 'The Narwhal',
 'The Tyee',
 'Toronto Star | Investigations',
 'Toronto Star | Editorials',
 'Toronto Star | Top Stories']

## Display Entries

`read` = None shows all entries

`read` = False shows only unread entries

`read` = True shows only read entries

In [ ]:
r.show(limit=5, read=None, mark_as_read=False)

╭─────────────────────────────────────────────────── The Tyee ────────────────────────────────────────────────────╮
│ Title: Glen Clark Is Open to Running BC Housing (in News)                                                       │
│ Date: Unknown                                                                                                   │
│                                                                                                                 │
│ The former premier left the private sector this week and says he’s offered to help David Eby’s government.      │
│                                                                                                                 │
╰────────────── ]8;id=686604;https://thetyee.ca/News/2023/01/06/Glen-Clark-Open-BC-Housing-David-Eby-Government/\https://thetyee.ca/News/2023/01/06/Glen-Clark-Open-BC-Housing-David-Eby-Government/]8;;\ ──────────────╯

╭─────────────────────────────────────────────────── The Tyee ────────────────────────────────────────────────────╮
│ Title: What Canadian Cities Can Learn from My 'Intense' Mumbai Neighbourhood (in Opinion)                       │
│ Date: Unknown                                                                                                   │
│                                                                                                                 │
│ Diversity, connection and a car-free life come when all the things you need, from fresh meals to doctors, are   │
│ steps away from your home.                                                                                      │
│                                                                                                                 │
╰─────────── ]8;id=771542;https://thetyee.ca/Opinion/2023/01/06/Canadian-Cities-Learn-Intense-Mumbai-Neighbourhood/\https://thetyee.ca/Opinion/2023/01/06/Canadian-Cities-Learn-Intense-Mumbai-Neighbourhood/]8;;\ ───────────╯

╭─────────────────────────────────────────────────── The Tyee ────────────────────────────────────────────────────╮
│ Title: BC Labour Board Rules Workers Can’t Refuse to Cross Some Picket Lines (in News)                          │
│ Date: Unknown                                                                                                   │
│                                                                                                                 │
│ The precedent-setting decision affects strikes by federally regulated employees in the province.                │
│                                                                                                                 │
╰───────────── ]8;id=273607;https://thetyee.ca/News/2023/01/06/BC-Labour-Board-Rules-Workers-Refuse-Picket-Lines/\https://thetyee.ca/News/2023/01/06/BC-Labour-Board-Rules-Workers-Refuse-Picket-Lines/]8;;\ ─────────────╯

╭─────────────────────────────────────────────────── The Tyee ────────────────────────────────────────────────────╮
│ Title: Telus Health Gets More Time in Extra-Billing Case (in News)                                              │
│ Date: Unknown                                                                                                   │
│                                                                                                                 │
│ The province is seeking an injunction to stop the corporation from charging patients for access to primary      │
│ care.                                                                                                           │
│                                                                                                                 │
╰────────────────────── ]8;id=244852;https://thetyee.ca/News/2023/01/06/Telus-Health-Extra-Billing-Case/\https://thetyee.ca/News/2023/01/06/Telus-Health-Extra-Billing-Case/]8;;\ ──────────────────────╯

╭─────────────────────────────────────────────────── The Tyee ────────────────────────────────────────────────────╮
│ Title: What Canada Needs to Understand about Albertans (in Analysis)                                            │
│ Date: Unknown                                                                                                   │
│                                                                                                                 │
│ Behind our charged politics are valid resentments. Here’s how we can have a productive conversation.            │
│                                                                                                                 │
╰──────────────── ]8;id=845160;https://thetyee.ca/Analysis/2023/01/05/What-Canada-Needs-Understand-Albertans/\https://thetyee.ca/Analysis/2023/01/05/What-Canada-Needs-Understand-Albertans/]8;;\ ─────────────────╯

## Search Entries

In [ ]:
r.search("rate", mark_as_read=False)

╭────────────────────────────────────────────── CBC | Business News ──────────────────────────────────────────────╮
│ Title: High office vacancy rates spell continuing trouble for shops dependent on worker foot traffic            │
│ Date: 2022-12-22                                                                                                │
│                                                                                                                 │
│ Businesses that depend on the flow of office workers are struggling as vacancy rates in office towers across    │
│ Canada continue to rise. Vancouver's office tower vacancy rate has tripled since 2019, while Montreal's has     │
│ doubled.                                                                                                        │
│                                                                                                                 │
╰────────── ]8;id=40257;https://www.cbc.ca/news/business/office-vacancy-rates-businesses-struggle-1.6692360?cmp=rss\https://www.cbc.ca/news/business/office-vacancy-rates-businesses-struggle-1.6692360?cmp=rss]8;;\ ──────────╯

╭────────────────────────────────────────────── CBC | Business News ──────────────────────────────────────────────╮
│ Title: Official inflation rate cools to 6.8% — but just about everything except gasoline keeps getting more     │
│ expensive                                                                                                       │
│ Date: 2022-12-21                                                                                                │
│                                                                                                                 │
│ Canada's inflation rate cooled to 6.8 per cent last month as prices for gasoline and furniture went down, but   │
│ the cost of food and rent went up.                                                                              │
│                                                                                                                 │
╰────────────────────── ]8;id=35479;https://www.cbc.ca/news/business/inflation-canada-1.6693441?cmp=rss\https://www.cbc.ca/news/business/inflation-canada-1.6693441?cmp=rss]8;;\ ──────────────────────╯

╭────────────────────────────────────────── Toronto Star | Top Stories ───────────────────────────────────────────╮
│ Title: ‘It’s choking me’: This homeowner’s mortgage payments have shot up by more than $1,000 a month. What     │
│ should he do?                                                                                                   │
│ Date: 2022-12-30                                                                                                │
│                                                                                                                 │
│ ‘I was naive. I’ve never experienced rate hikes like this before,’ homeowner says, adding he plans to switch to │
│ a five-year fixed rate.                                                                                         │
│                                                                                                                 │
╰─ ]8;id=906563;https://www.thestar.com/business/personal_finance/2022/12/30/the-last-thing-you-want-to-do-it-sell-your-home-as-variable-mortgages-rates-go-up-cutting-back-significantly-on-expenses-is-required-short-term-experts-say.html\https://www.thestar.com/business/personal_finance/2022/12/30/the-last-thing-you-want-to-do-it-sell-your-home-a]8;;\─╯

╭────────────────────────────────────────────── CBC | Business News ──────────────────────────────────────────────╮
│ Title: Canada's economy added 104,000 jobs in December, blowing well past expectations                          │
│ Date: 2023-01-06                                                                                                │
│                                                                                                                 │
│ Canada's economy added 104,000 jobs in December as the unemployment rate fell slightly to 5.0 per cent,         │
│ Statistics Canada said Friday. But the gain wasn't reflected in working hours, and a hot labour market could    │
│ mean another rate hike from the central bank in January.                                                        │
│                                                                                                                 │
╰──────────────────── ]8;id=883438;https://www.cbc.ca/news/business/canada-jobs-december-1.6705569?cmp=rss\https://www.cbc.ca/news/business/canada-jobs-december-1.6705569?cmp=rss]8;;\ ────────────────────╯

╭───────────────────────────────────────────────── The Guardian ──────────────────────────────────────────────────╮
│ Title: We are ignoring Covid case rises once again | Letters                                                    │
│ Date: 2022-12-29                                                                                                │
│                                                                                                                 │
│ The decision to stop reporting the Covid growth rate is the latest in a series of attempts to limit the         │
│ available information about the current spread, writes Dr Brian Ramsden, while Susan Treagus has been keeping a │
│ Covid diaryWith a predictable adherence to the principle of burying bad news, the government’s UK Health        │
│ Security Agency announced on the Friday before Christmas that it is now ceasing to publish the reproduction     │
│ number – the R value – and the growth rate for Covid-19 in the UK (UK to stop publishing Covid modelling data,  │
│ 25 December).This decision was taken at a time when the R value and the growth rate both show that the          │
│ prevalence of Covid is increasing in the UK. This is just the latest in a series of attempts to minimise the    │
│ availability of information about Covid. Data about tests is virtually nonexistent, because tests are no longer │
│ freely available; contact tracing has ceased; and daily reporting of infections stopped when people were told   │
│ there was no longer a need to use masks or to self-isolate. Continue reading...                                 │
│                                                                                                                 │
╰─────────── ]8;id=860048;https://www.theguardian.com/world/2022/dec/29/we-are-ignoring-covid-case-rises-once-again\https://www.theguardian.com/world/2022/dec/29/we-are-ignoring-covid-case-rises-once-again]8;;\ ───────────╯

╭───────────────────────────────────────────────── The Guardian ──────────────────────────────────────────────────╮
│ Title: Inflation, tech plunge and ‘crypto winter’: global stock markets in 2022                                 │
│ Date: 2022-12-30                                                                                                │
│                                                                                                                 │
│ Shares tumble and government bonds hammered by central banks’ interest rate risesFTSE 100 ends 2022 slightly up │
│ despite global turmoilInvestors are reeling from the worst year in global financial markets since the 2008      │
│ financial crisis, as inflation forced central banks around the world to raise interest rates and put an end to  │
│ the supply of cheap money that has fuelled a golden decade for investors.Global stocks have lost about a fifth  │
│ of their value during the last year, as the “everything bubble” that inflated during the Covid-19 pandemic      │
│ burst, sending tech shares and crypto assets tumbling. Continue reading...                                      │
│                                                                                                                 │
╰─ ]8;id=91454;https://www.theguardian.com/business/2022/dec/30/inflation-tech-plunge-crypto-global-stock-markets-2022-shares-bonds-interest-rates\https://www.theguardian.com/business/2022/dec/30/inflation-tech-plunge-crypto-global-stock-markets-2022-shares]8;;\─╯

╭───────────────────────────────────────── Toronto Star | Investigations ─────────────────────────────────────────╮
│ Title: Toronto parents less likely to see reduced daycare fees than almost anywhere else in Ontario             │
│ Date: 2022-12-22                                                                                                │
│                                                                                                                 │
│ For-profit daycares opted out of $10/day child care program at much higher rate than non-profits, a Star        │
│ analysis found.                                                                                                 │
│                                                                                                                 │
╰─ ]8;id=131531;https://www.thestar.com/news/investigations/2022/12/22/toronto-parents-less-likely-to-see-reduced-daycare-fees-than-almost-anywhere-else-in-ontario.html\https://www.thestar.com/news/investigations/2022/12/22/toronto-parents-less-likely-to-see-reduced-daycare-fees]8;;\─╯

╭───────────────────────────────────────────────── The Guardian ──────────────────────────────────────────────────╮
│ Title: UK inflation will fall in 2023 but energy bills and taxes will rise as house prices drop. Happy new year │
│ | Larry Elliott                                                                                                 │
│ Date: 2023-01-01                                                                                                │
│                                                                                                                 │
│ Britain’s recession, however, will be relatively mild, although such forecasting is a mug’s gameWar in Europe.  │
│ Political turmoil. A stagnant economy. Soaring energy bills. The highest inflation rate in more than four       │
│ decades. Higher interest rates. That was the story of 2022. Truly, the year just gone was – in the words of the │
│ late queen – an annus horribilis.So 2023 will have to go some to match its predecessor for drama. The bookies   │
│ would, for example, give you relatively long odds on it being another year of three prime ministers and four    │
│ chancellors of the exchequer. Continue reading...                                                               │
│                                                                                                                 │
╰─ ]8;id=277481;https://www.theguardian.com/business/2023/jan/01/uk-inflation-energy-bills-taxes-house-prices-happy-new-year\https://www.theguardian.com/business/2023/jan/01/uk-inflation-energy-bills-taxes-house-prices-happy-new-year]8;;\ ──╯

╭──────────────────────────────────────────── Science | The Guardian ─────────────────────────────────────────────╮
│ Title: UK to stop publishing Covid modelling data                                                               │
│ Date: 2022-12-25                                                                                                │
│                                                                                                                 │
│ R range and growth rate, which during the height of the pandemic was published weekly in England, deemed ‘no    │
│ longer necessary’The UK Health Security Agency will stop publishing modelling data on coronavirus in early      │
│ January.The chief data scientist, Dr Nick Watkins, said the publication of this specific data is “no longer     │
│ necessary” as the country is living with Covid thanks to vaccines and therapeutics. Continue reading...         │
│                                                                                                                 │
╰─────────── ]8;id=884222;https://www.theguardian.com/world/2022/dec/25/uk-to-stop-publishing-covid-modelling-data\https://www.theguardian.com/world/2022/dec/25/uk-to-stop-publishing-covid-modelling-data]8;;\ ────────────╯

╭───────────────────────────────────────────────── The Guardian ──────────────────────────────────────────────────╮
│ Title: ‘Groundhog year’: UK disposable incomes to fall by 3.8% in 2023                                          │
│ Date: 2022-12-30                                                                                                │
│                                                                                                                 │
│ The Resolution Foundation thinktank forecasts a drop in living standards for a second year as the cost of       │
│ living crisis deepensHouseholds are facing a “groundhog year” in 2023, as soaring gas bills and planned tax     │
│ rises squeeze disposable incomes and send living standards tumbling for the second year running.Higher mortgage │
│ costs as fixed-rate loans come to an end and new deals are negotiated will add to the financial burden already  │
│ felt by millions of households reeling from the worst fall in living standards in a century. Continue           │
│ reading...                                                                                                      │
│                                                                                                                 │
╰── ]8;id=709920;https://www.theguardian.com/business/2022/dec/30/groundhog-year-uk-disposable-incomes-to-fall-by-38-in-2023\https://www.theguardian.com/business/2022/dec/30/groundhog-year-uk-disposable-incomes-to-fall-by-38-in-2023]8;;\ ──╯

## Tag Entry

In [ ]:
test = list(r._reader.get_entries())

In [ ]:
r._reader.set_tag(test[0], "foobar")

In [ ]:
list(r._reader.get_tags(test[0]))

[('foobar', None)]

In [ ]:
# We added a method to just return the tag key:
r._get_tags(test[0])

['foobar']

## Remove tag from entry

In [ ]:
r._reader.delete_tag(test[0], "foobar")

In [ ]:
list(r._get_tags(test[0]))

[]

## Mark as Important/Unimportant

In [ ]:
r._reader.mark_entry_as_important(test[0])
r._reader.mark_entry_as_unimportant(test[0])

## Mark as Read / Unread

In [ ]:
r._reader.mark_entry_as_read(test[0])
r._reader.mark_entry_as_unread(test[0])

## Automatically mark entries as read

When updating feeds, entries with titles matching these strings will automatically be marked as read

In [ ]:
for i in ["Musk", "Apple", "Bezos", "Google", "Samsung", "iPhone", "iPad", "guardian.com/sport/", "guardian.com/football/", "thestar.com/sports"]:
    r._add_to_blacklist(i)

In [ ]:
r._title_blacklist

['Musk',
 'Apple',
 'Bezos',
 'Google',
 'Samsung',
 'iPhone',
 'iPad',
 'Premier League',
 'NHL',
 'NFL',
 'NBA',
 'guardian.com/sport/',
 'guardian.com/football/',
 'thestar.com/sports']

In [ ]:
#r.update()

## Automatically mark entries as important

In [ ]:
for i in ["interest rate", "Bank of Canada", "housing market"]:
    r._add_to_whitelist(i)

In [ ]:
#r.update()

## Low level access to data

In [ ]:
entries = r._get_entries(limit=1)
[i for i in entries][0]

Entry(id='https://thetyee.ca/News/2023/01/06/Glen-Clark-Open-BC-Housing-David-Eby-Government/', updated=None, title='Glen Clark Is Open to Running BC Housing (in News)', link='https://thetyee.ca/News/2023/01/06/Glen-Clark-Open-BC-Housing-David-Eby-Government/', author='Andrew MacLeod', published=None, summary='The former premier left the private sector this week and says he’s offered to help David Eby’s government.', content=(), enclosures=(Enclosure(href='https://thetyee.ca/News/2023/01/06/GlenClarkNDPConvention.jpg', type='image/jpeg', length=None),), read=False, read_modified=datetime.datetime(2023, 1, 7, 15, 5, 8, 100984, tzinfo=datetime.timezone.utc), important=False, important_modified=datetime.datetime(2023, 1, 7, 15, 5, 6, 970716, tzinfo=datetime.timezone.utc), added=datetime.datetime(2023, 1, 7, 15, 2, 53, 727599, tzinfo=datetime.timezone.utc), added_by='feed', last_updated=datetime.datetime(2023, 1, 7, 15, 2, 53, 727599, tzinfo=datetime.timezone.utc), original_feed_url='https

## CLI

In [ ]:
# | export


def main():
    fire.Fire(PyNewsReader)

In [ ]:
# | hide

# To Do:
# * User interface

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()